In [1]:
import re
import pandas as pd
from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

import pickle
tfidf = pickle.load(open("tfidf.pickle",'rb'))
res = pickle.load(open("transform.pickle",'rb'))
grid_cv = pickle.load(open("grid_cv.pickle",'rb'))

In [2]:
df = pd.read_csv('sentiment.csv')
df['1'].value_counts()

1    45135
0    32418
Name: 1, dtype: int64

In [3]:
best = grid_cv.best_estimator_
prd = best.predict(res)
from sklearn.metrics import accuracy_score

print('score : ', round(accuracy_score(df['1'], prd), 4))

score :  0.9994


In [6]:
from flask import Flask, request
app = Flask(__name__)
@app.route('/')
def wellcome():
    return "HelloWorld"
@app.route('/shutdown')
def shutdown():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()
    return "shutdown"
@app.route('/work', methods=['get'])
def work():
    key_dict = ['5063a0fae26d87acc2dd2fca8a374c9454c890ac0f650b2ba23ba1162991b63a', 'c05f9f2e8fcb0ce1f01d7ecddc04b37d10639df01b1a2db7ce29e8647fd53235', 'dd1caa78827e84ced4e89a6110534ec276070415054834ee1cdd2e729d6dd5ff','dd1caa78827e84ced4e89a6110534ec276070415054834ee1cdd2e729d6dd5ff','820963ca3e2bf4430ab66848a1a80d30646baa80a2129486b3119baa1cd44829','1b5394c88f45ee017a2d5d15f7e9294faea9d36f02221f88c5363b31ead17aae','b4131552f0f8f391665dfa8c4c5180045e22a4281891a5a008412898fad87c93']

    import requests
    uid = request.args.get('uid')
    data = request.args.get('text')
    pid = request.args.get('pid')
    temp = data
    if pid is not None and data is not None and uid is not None:

        scan_url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', data)
        data = re.compile(r'[가-힣]+').findall(data)
        data = [" ".join(data)]
        data_tfidf = tfidf.transform(data)
        data_predict = best.predict(data_tfidf)
        rq = requests.post('http://localhost:8080/ai/response', data={'uid':uid, '_confirm':data_predict,'message':temp,'pid':pid}).json()
        print(rq)
        if scan_url!=[]:
            scan_url = scan_url[0]
            urlcheck = requests.post('http://localhost:8080/ai/url', data={'url':scan_url}).json()
            url_confirm = str(urlcheck.get('data')['confirm'])

            if url_confirm=="-1":
                for my_apikey in key_dict:
                    try:
                        url_confirm=0
                        url = 'https://www.virustotal.com/vtapi/v2/url/scan'
                        params = {'apikey': my_apikey, 'url': scan_url}
                        response_scan = requests.post(url, data=params)
                        result_scan = response_scan.json()
                        scan_id = result_scan['scan_id']  # 결과를 출력을 위해 scan_id 값 저장
                        url_report = 'https://www.virustotal.com/vtapi/v2/url/report'
                        url_report_params = {'apikey': my_apikey, 'resource': scan_id}
                        response_report = requests.get(url_report, params=url_report_params)
                        report = response_report.json()  # 결과 값을 report에 json형태로 저장
                        report_scans = report.get('scans')  # scans 값 저장
                        report_scans_vendors = list(report['scans'].keys())  # Vendor 저장
                        for vendor in report_scans_vendors:
                            outputs_result = report_scans[vendor].get('result')
                            if outputs_result != 'clean site':
                                if outputs_result != 'unrated site':
                                    url_confirm = 1
                        requests.post('http://localhost:8080/ai/insert_url', data={'pid':pid, '_confirm':url_confirm,'url':scan_url}).json()
                        break
                    except:
                        print('key error : ' + my_apikey)
            return {'message_confirm':str(data_predict[0]),'qid':str(rq.get('data')['qid']), 'url':scan_url, "url_confirm":url_confirm}
        else:
            return {'message_confirm':str(data_predict[0]),'qid':str(rq.get('data')['qid']), 'url':scan_url, "url_confirm":0}
    else:
        return {'parameter error'}

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


{'message': '200', 'data': '404'}


[2022-09-22 19:32:43,068] ERROR in app: Exception on /work [GET]
Traceback (most recent call last):
  File "C:\Users\0\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\0\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\0\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\0\anaconda3\lib\http\client.py", line 1377, in getresponse
    response.begin()
  File "C:\Users\0\anaconda3\lib\http\client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "C:\Users\0\anaconda3\lib\http\client.py", line 281, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\0\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv

{'message': '200', 'data': {'qid': 445}}


...

{'message': '200', 'data': {'qid': 449}}


127.0.0.1 - - [22/Sep/2022 19:55:26] "GET /work?text=안녕%20반가워%20%20https%3A%2F%2Fbprhost.com%2Fdigital%2Faccedi%2F%201231231232%20으휴...%20그만해라%21&uid=1&pid=1 HTTP/1.1" 200 -


{'message': '200', 'data': {'qid': 450}}


127.0.0.1 - - [22/Sep/2022 19:55:40] "GET /work?text=안녕%20반가워%20%20https%3A%2F%2Fbprhost.com%2Fdigital%2Faccedi%2F%201231231232%20으휴...%20그만해라%21&uid=1&pid=1 HTTP/1.1" 200 -


{'message': '200', 'data': {'qid': 451}}
